In [65]:
import pandas as pd
import numpy as np
import hdbscan
import umap

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from hdbscan import HDBSCAN

from collections import Counter

In [2]:
data = pd.read_csv("./coursera_reviews_processed.csv")

In [3]:
data.dropna(subset=["processed_reviews"], inplace=True)
data['length'] = data["processed_reviews"].apply(lambda x : len(x.split(" ")))
data = data[data['length'] >= 20]

In [4]:
len(data)

55540

In [5]:
# average length of reviews
np.mean(data["processed_reviews"].apply(lambda x : len(x.split(" "))))

33.79211379186172

In [6]:
num_features = 200
vect = TfidfVectorizer(max_features=num_features)
vect.fit(data["processed_reviews"])
reviews_dtm = vect.transform(data["processed_reviews"])
top_words = vect.get_feature_names()

C:\Users\13064\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [24]:
lda = LatentDirichletAllocation(n_components=4, max_iter=100, random_state=42, n_jobs=-1)
lda_prob = lda.fit_transform(reviews_dtm)
lda_result = np.argmax(np.array(lda_prob), axis=1)
data['lda_result'] = lda_result
topic_words_weight = lda.components_

In [78]:
topic_id = 0
for words_weight in topic_words_weight:
    word_dict = [(tw, round(ww, 1)) for tw, ww in zip(top_words, words_weight)]
    word_dict = sorted(word_dict, key=lambda x: x[1], reverse=True)
    word_dict = word_dict[:15]
    print('topic%d:' % (topic_id), word_dict, "\n")
    topic_id += 1

topic0: [('cours', 2027.3), ('assign', 1400.8), ('data', 1357.2), ('good', 1189.9), ('video', 1153.0), ('would', 1014.6), ('use', 939.2), ('like', 910.2), ('week', 906.5), ('time', 854.2), ('lectur', 833.8), ('question', 828.7), ('materi', 799.3), ('could', 782.3), ('bit', 742.3)] 

topic1: [('cours', 2127.0), ('thank', 1248.1), ('learn', 1202.7), ('python', 944.8), ('realli', 900.9), ('program', 884.6), ('dr', 881.1), ('great', 811.2), ('languag', 768.5), ('teach', 682.4), ('coursera', 668.1), ('way', 636.2), ('love', 634.8), ('much', 626.9), ('professor', 622.9)] 

topic2: [('deep', 871.2), ('network', 705.2), ('learn', 699.2), ('andrew', 691.9), ('cours', 657.1), ('step', 624.9), ('ai', 580.5), ('neural', 562.2), ('build', 533.0), ('machin', 507.1), ('project', 435.4), ('understand', 373.4), ('concept', 322.1), ('great', 278.8), ('basic', 250.4)] 

topic3: [('cours', 2193.7), ('learn', 1396.9), ('thank', 985.9), ('help', 972.1), ('life', 874.0), ('use', 828.7), ('great', 769.9), ('i

In [72]:
u_model = umap.UMAP(n_components=5)
reduce_reviews = u_model.fit_transform(reviews_dtm.toarray())
cluster = HDBSCAN(min_cluster_size=3, min_samples=2000, metric='euclidean', prediction_data=True, gen_min_span_tree=True)
predict_cluster_result = cluster.fit_predict(reduce_reviews)
data["cluster_result"] = predict_cluster_result

OSError: [WinError 1450] 系统资源不足，无法完成请求的服务。

In [13]:
label_num_dict = dict(Counter(cluster.labels_))
for k in label_num_dict:
    if label_num_dict[k] > 1000:
        print(k)

0
-1
1


In [14]:
data[data['cluster_result'] == 0]['processed_reviews']

2         recommend wide cours reach import inform weste...
5         far best cours html css javascript great instr...
8         cours brilliant succinct scientif illustr prov...
15        part predict analyt case studi actual good two...
18        think cours everyon industrialis societi eat e...
                                ...                        
289814    cours realli explain core principl ux deal rea...
289817    final exam question wrong none select answer c...
289819    great content mathemat idea well explain probl...
289822    realli like cours teacher explain well learnt ...
289824    good cours overal last two week assign littl b...
Name: processed_reviews, Length: 45287, dtype: object

In [15]:
data.to_csv("topic_result.csv", encoding='utf-8')